In [9]:
from tqdm import tqdm
import random as rd
import copy
from itertools import combinations
import numpy as np
from os import listdir
from os.path import isfile, join
from os import path
import argparse
import csv
import time
from pathlib import Path


In [35]:
import numpy as np

__all__ = ['QAData', 'QAReader', 'init_solution']


def init_solution(n):
    return np.random.permutation(np.arange(n))


class QAData:
    def __init__(self, n, distances, flows):
        self.n = n
        self.distances = distances
        self.flows = flows

    def compute_cost(self, solution, **kwargs):
        cost = 0
        for i in range(self.n):
            for j in range(self.n):
                dist = self.distances[solution[i]][solution[j]]
                flow = self.flows[i][j]
                cost += flow * dist

        if 'method' not in kwargs.keys():
            return cost
        elif kwargs['method'] == 'guided':
            augmented_part = 0
            mu = kwargs['mu']
            indicator_func = kwargs['indicator']
            penalty = kwargs['penalty']
            for u in range(self.n):
                for v in range(self.n):
                    augmented_part += indicator_func[u][v] * penalty[u][v]
            return cost + mu * augmented_part


class QAReader:
    def __init__(self):
        pass

    def __call__(self, path):
        with open(path, "r") as f:
            n = int(f.readline().strip())
            distances, flows = np.empty((n, n)), np.empty((n, n))
            for i in range(n):
                flows[i] = (list(map(int, f.readline().split())))
            _ = f.readline()
            for j in range(n):
                distances[j] = (list(map(int, f.readline().split())))
        return QAData(n, distances, flows)

In [36]:
class IteratedLocalSearch:
    def __init__(self, data, method, verbose, n_iter):
        self.data = data
        self.verbose = verbose
        self.method = method
        self.iter_amount = n_iter
        self.solution = init_solution(self.data.n)
        self.current_cost = self.data.compute_cost(self.solution)
        self.solver = LocalSearch(self.data, method, False, int(self.iter_amount * 0.1))
        self.cost_history = []

    def perturbation(self):
        k = rd.randint(2, self.data.n)
        indexes = np.random.choice(np.arange(self.data.n), k, replace=False)
        shuffled_indexes = np.random.permutation(indexes)
        new_solution = self.solution.copy()
        new_solution[indexes] = self.solution[shuffled_indexes]
        return new_solution

    def acceptance_criterion(self, new_solution):
        if self.data.compute_cost(self.solution) > self.data.compute_cost(new_solution):
            self.solution = new_solution

    def __call__(self):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')
        self.solution, cost = self.solver(self.solution)
        for _ in tqdm(range(self.iter_amount)):
            new_solution, cost = self.solver(self.perturbation())
            self.cost_history.append(cost)
            self.acceptance_criterion(new_solution)
        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

In [37]:
class LocalSearch:
    def __init__(self, data, method, verbose, n_iter, solution=None):
        self.data = data
        self.method = method
        self.verbose = verbose
        self.iter_amount = n_iter
        self.solution = init_solution(self.data.n)
        self.current_cost = self.data.compute_cost(self.solution)
        self.patience = int(0.2 * self.iter_amount)
        self.improved = True
        self.no_improvements = 0

    def stohastic_2_opt(self, **kwargs):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')

        for _ in tqdm(range(self.iter_amount), disable=not self.verbose):
            if not self.improved:
                self.no_improvements += 1
            if self.no_improvements == self.patience:
                break
            best_solution = self.solution
            self.improved = False
            for _ in range(int(0.2 * self.iter_amount)):
                ind_left = rd.randint(0, self.data.n - 1)
                ind_right = rd.randint(ind_left + 1, self.data.n)
                tmp_solution = self.solution.copy()
                tmp_solution[ind_left:ind_right] = tmp_solution[ind_left:ind_right][::-1]
                cost = self.data.compute_cost(tmp_solution, **kwargs)
                if cost < self.current_cost:
                    self.improved = True
                    self.no_improvements = 0
                    self.current_cost = cost
                    best_solution = tmp_solution
            self.solution = best_solution
        final_cost = self.data.compute_cost(self.solution, **kwargs)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

    def count_delta(self, r, s):
        diff = 0
        pi = self.solution
        for k in range(self.data.n):
            if k != r and k != s:
                diff += (self.data.flows[k, r] + self.data.flows[r, k]) * \
                        (self.data.distances[pi[s], pi[k]] - self.data.distances[pi[r], pi[k]]) + \
                        (self.data.flows[k, s] + self.data.flows[s, k]) * \
                        (self.data.distances[pi[r], pi[k]] - self.data.distances[pi[s], pi[k]])
        return diff

    def count_delta_with_previous(self, previous, u, v, r, s):
        pi = self.solution
        return previous + 2 * (self.data.flows[r, u] - self.data.flows[r, v] + self.data.flows[s, v] - self.data.flows[s, u]) * \
                          (self.data.distances[pi[s], pi[u]] - self.data.distances[pi[s], pi[v]] + self.data.distances[pi[r], pi[v]] - self.data.distances[pi[r], pi[u]])

    def first_with_delta(self):
        if self.verbose:
            print('Start cost {}'.format(self.current_cost))

        dont_look_bits = np.zeros(self.data.n, dtype=np.bool)
        for _ in tqdm(range(self.iter_amount), disable=not self.verbose):
            comb = combinations(np.arange(self.data.n, dtype=np.int32), 2)
            if not self.improved:
                self.no_improvements += 1
            if self.no_improvements == self.patience or np.all(dont_look_bits):
                break
            self.improved = False
            curr_city = 0
            counter = 0
            for opt in comb:
                if dont_look_bits[opt[0]] or dont_look_bits[opt[1]]:
                    continue
                opt = list(opt)
                if curr_city != opt[0] and counter == self.data.n - 1 - curr_city:
                    dont_look_bits[curr_city] = 1
                    curr_city += 1
                    counter = 0
                elif curr_city != opt[0]:
                    curr_city += 1
                    counter = 0
                delta = self.count_delta(opt[0], opt[1])
                if delta < 0:
                    self.improved = True
                    self.no_improvements = 0
                    self.current_cost += delta
                    self.solution[opt] = self.solution[opt][::-1]
                    break
                elif curr_city == opt[0]:
                    counter += 1

        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost


    def first_improvement(self):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')

        dont_look_bits = np.zeros(self.data.n, dtype=np.bool)
        for _ in tqdm(range(self.iter_amount), disable=not self.verbose):
            comb = combinations(np.arange(self.data.n, dtype=np.int32), 2)
            if not self.improved:
                self.no_improvements += 1
            if self.no_improvements == self.patience or np.all(dont_look_bits):
                break
            self.improved = False
            curr_city = 0
            counter = 0
            for opt in comb:
                if dont_look_bits[opt[0]] or dont_look_bits[opt[1]]:
                    continue
                opt = list(opt)
                tmp_solution = self.solution.copy()
                tmp_solution[opt] = tmp_solution[opt][::-1]
                cost = self.data.compute_cost(tmp_solution)
                if curr_city != opt[0] and counter == self.data.n - 1 - curr_city:
                    dont_look_bits[curr_city] = 1
                    curr_city += 1
                    counter = 0
                elif curr_city != opt[0]:
                    curr_city += 1
                    counter = 0
                if cost < self.current_cost:
                    self.improved = True
                    self.no_improvements = 0
                    self.current_cost = cost
                    self.solution = tmp_solution
                    break
                elif curr_city == opt[0]:
                    counter += 1


        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

    def best_improvement(self):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')
        previous_opt = None
        previous_delta = None
        for _ in tqdm(range(self.iter_amount), disable=not self.verbose):
            comb = list(combinations(np.arange(self.data.n, dtype=np.int32), 2))
            if not self.improved:
                self.no_improvements += 1
            if self.no_improvements == self.patience:
                break
            self.improved = False
            min_delta = previous_delta if previous_delta is not None else 0
            optimal_opt = None
            for opt in comb:
                opt = list(opt)
                if previous_opt is not None and not opt == previous_opt:
                    delta = self.count_delta_with_previous(previous_delta, *opt, *previous_opt)
                elif opt == previous_opt:
                    continue
                else:
                    delta = self.count_delta(*opt)
                if delta < min_delta:
                    min_delta = delta
                    optimal_opt = opt
            if optimal_opt is not None:
                self.solution[optimal_opt] = self.solution[optimal_opt][::-1]
                self.improved = True
                previous_opt = optimal_opt
                previous_delta = min_delta
            else:
                self.improved = False
        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

    def __call__(self, solution=None, **kwargs):
        if solution is not None:
            self.solution = solution
        if self.method == '2-opt':
            return self.stohastic_2_opt(**kwargs)
        elif self.method == 'first-improvement':
            return self.first_improvement()
        elif self.method == 'first-with-delta':
            return self.first_with_delta()
        else:
            return self.best_improvement()

In [38]:
best_known = {
    'tai20a': 703482,
    'tai40a': 3139370,
    'tai60a': 7205962,
    'tai80a': 13499184,
    'tai100a': 21044752,
}

In [39]:
            n_iter = 100
            file_res = open("C:\\Users\\Vlad\\Desktop\\Lab4\\result.csv", 'w')
            columns_names = ['File name', 'Method', 'Best known', 'Result', 'Time']
            writer = csv.DictWriter(file_res, fieldnames=columns_names)
            writer.writeheader()
            reader = QAReader()
            benchmarks = [f for f in listdir('C:\\Users\\Vlad\\Desktop\\Lab4\\Data') if isfile(join('C:\\Users\\Vlad\\Desktop\\Lab4\\Data', f))]
            for file in benchmarks:
                data = reader(path.join('C:\\Users\\Vlad\\Desktop\\Lab4\\Data', file))
                algorithms = [(name, f(data, 'first-with-delta', True, n_iter)) for name, f in [("local search", LocalSearch), ("iterated local search", IteratedLocalSearch)] if callable(f)]
                for name, algorithm in algorithms:
                    Path(f"./{name}").mkdir(parents=True, exist_ok=True)
                    print(f'{name} working on {file}')
                    start_time = time.time()
                    solution, final_cost = algorithm()
                    work_time = round(time.time() - start_time, 4)
                    np.savetxt(f'./{name}/{file}.sol', solution.reshape(1, solution.shape[0]), fmt='%d')
                    writer.writerow({
                        'File name': file,
                        'Method': name,
                        'Best known': best_known[file],
                        'Result': final_cost,
                        'Time': work_time
                    })

<ipython-input-37-3eb63bfa0099>:61: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dont_look_bits = np.zeros(self.data.n, dtype=np.bool)
 38%|█████████████████████████████▋                                                | 38/100 [00:00<00:00, 351.85it/s]

local search working on tai100a
Start cost 24152846.0


  3%|██▍                                                                             | 3/100 [00:00<00:04, 21.90it/s]

Final cost 23045840.0
iterated local search working on tai100a
Starting value of cost func is 24105932.0


 18%|██████████████                                                                | 18/100 [00:00<00:00, 165.14it/s]

...

Final cost 7944538.0
local search working on tai80a
Start cost 15706290.0


  3%|██▍                                                                             | 3/100 [00:00<00:04, 23.81it/s]

Final cost 14700210.0
iterated local search working on tai80a
Starting value of cost func is 15736964.0


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.26it/s]

Final cost 14617158.0


In [47]:
print('Local Search')
for file in listdir('C:\\Users\\Vlad\\local search'):
    sol = open(f'C:\\Users\\Vlad\\local search\\{file}', 'r').read()
    print(f'Name: {file}. Solution {sol}')

Local Search
Name: tai100a.sol. Solution 31 74 52 69 21 57 66 23 48 28 81 71 56 24 18 77 46 67 3 85 75 26 34 2 98 12 84 96 4 15 39 78 20 10 68 17 9 13 62 94 41 79 89 72 50 91 58 65 29 30 60 0 44 99 19 1 59 11 55 7 8 36 32 45 64 42 35 86 53 54 38 49 82 47 63 37 76 16 33 80 43 25 5 61 83 14 88 90 51 70 93 22 95 6 40 27 73 87 97 92

Name: tai20a.sol. Solution 11 9 7 3 14 13 12 18 8 4 2 6 1 15 17 16 19 5 10 0

Name: tai40a.sol. Solution 25 28 20 12 36 2 16 23 17 10 29 13 11 38 37 14 4 0 1 7 21 5 6 33 27 19 3 32 31 15 24 35 9 8 22 30 18 39 26 34

Name: tai60a.sol. Solution 46 19 27 58 38 30 20 49 42 5 48 26 56 34 2 37 43 23 41 44 24 28 29 35 10 36 7 53 14 32 25 22 8 17 6 54 13 11 18 45 4 33 39 50 1 9 15 3 16 59 51 55 52 21 57 47 31 40 12 0

Name: tai80a.sol. Solution 21 65 22 46 70 75 35 52 53 51 17 55 28 71 40 7 19 42 47 20 57 63 78 54 14 4 60 31 45 56 66 5 3 30 38 0 16 74 29 32 68 11 6 62 25 67 36 1 73 50 26 49 44 27 23 76 18 79 43 12 37 2 61 72 39 24 10 34 59 13 64 9 41 69 48 58 33 8 77 

In [48]:
print('Iterated Local Search')
for file in listdir('C:\\Users\\Vlad\\iterated local search'):
    sol = open(f'C:\\Users\\Vlad\\iterated local search\\{file}', 'r').read()
    print(f'Name: {file}. Solution {sol}')

Iterated Local Search
Name: tai100a.sol. Solution 66 72 67 65 69 58 80 91 63 1 38 61 68 56 36 97 27 96 9 79 85 20 55 90 73 44 92 2 93 41 31 19 50 40 59 75 29 39 51 99 30 35 57 78 45 18 5 15 3 52 47 6 82 34 81 76 7 24 22 8 64 43 62 26 98 25 17 71 84 33 4 86 13 87 70 95 74 23 32 89 94 83 11 88 10 16 53 21 28 54 77 46 0 42 60 12 48 14 49 37

Name: tai20a.sol. Solution 13 8 11 9 1 0 10 15 19 16 14 3 4 12 17 18 6 5 7 2

Name: tai40a.sol. Solution 29 23 31 27 34 37 33 2 28 9 6 0 11 14 5 21 17 30 36 32 25 7 13 38 22 18 20 12 19 24 26 3 4 10 1 15 39 16 35 8

Name: tai60a.sol. Solution 55 1 35 17 39 2 43 8 38 48 49 51 30 31 11 21 56 27 37 41 58 34 52 29 25 46 44 3 24 20 7 18 19 36 4 33 42 57 47 22 0 40 59 23 50 5 10 16 6 53 54 13 12 45 9 28 15 14 32 26

Name: tai80a.sol. Solution 38 79 42 40 39 46 37 26 44 77 18 2 61 65 64 20 51 47 9 22 14 29 34 15 76 62 45 4 11 70 52 3 78 31 56 50 33 35 7 74 49 21 58 32 19 36 16 43 30 66 67 5 13 59 23 54 28 48 57 68 17 71 10 73 12 41 24 6 72 0 27 75 63 60 25 5